Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
**For questions/comments/improvements, email nathan.kelber@ithaka.org.**<br />
![CC BY License Logo](https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png)
____

# Finding Significant Words Using TF/IDF

**Description of methods in this notebook:**
This [notebook](https://docs.tdm-pilot.org/key-terms/#jupyter-notebook) shows how to discover significant words in your [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor) and/or [Portico](https://docs.tdm-pilot.org/key-terms/#portico) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) using [Python](https://docs.tdm-pilot.org/key-terms/#python). The method for finding significant terms is [tf-idf](https://docs.tdm-pilot.org/key-terms/#tf-idf).  The following processes are described:

* Converting your [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor)/[Portico](https://docs.tdm-pilot.org/key-terms/#portico) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) into a Python list
* Filtering out articles from your [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) depending on a flexible set of rules
* Writing a helper function to help clean up a single [token](https://docs.tdm-pilot.org/key-terms/#token)
* Cleaning each document of your dataset, one [token](https://docs.tdm-pilot.org/key-terms/#token) at a time
* Using htrc_dictionary to remove words with poor [OCR](https://docs.tdm-pilot.org/key-terms/#ocr)
* Creating a [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary)
* Creating a [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) [bag of words]((https://docs.tdm-pilot.org/key-terms/#bag-of-words) [corpus](https://docs.tdm-pilot.org/key-terms/#corpus)
* Computing the most significant words in your [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) using [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) implementation of [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf)

**Difficulty:** Intermediate

**Knowledge Required:** 
* [Python Basics I](./0-python-basics-1.ipynb)
* [Python Basics II](./0-python-basics-2.ipynb)
* [Python Basics III](./0-python-basics-3.ipynb)

**Knowledge Recommended:**
* [Exploring Metadata](./1-metadata.ipynb)
* A familiarity with [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) is helpful but not required.

**Completion time:** 90 minutes

**Data Format:** [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor)/[Portico](https://docs.tdm-pilot.org/key-terms/#portico) [JSON Lines (.jsonl)](https://docs.tdm-pilot.org/key-terms/#jsonl)

**Libraries Used:**
* **[json](https://docs.tdm-pilot.org/key-terms/#json-python-library)** to convert our dataset from json lines format to a Python list
* **[gensim](https://docs.tdm-pilot.org/key-terms/#gensim)** to help compute the [tf-idf](https://docs.tdm-pilot.org/key-terms/#tf-idf) calculation
____

## What is "Term Frequency- Inverse Document Frequency" (TF-IDF)?

[TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) is used in [machine learning](https://docs.tdm-pilot.org/key-terms/#machine-learning) and [natural language processing](https://docs.tdm-pilot.org/key-terms//#nlp) for measuring the significance of particular terms for a given document. It consists of two parts that are multiplied together:

1. Term Frequency- A measure of how many times a given word appears in a document
2. Inverse Document Frequency- A measure of how many times the same word occurs in other documents within the corpus

If we were to merely consider [word frequency](https://docs.tdm-pilot.org/key-terms/#word-frequency), the most frequent words would be common [function words](https://docs.tdm-pilot.org/key-terms/#function-words) like: "the", "and", "of". We could use a [stopwords list](https://docs.tdm-pilot.org/key-terms/#stop-words) to remove the common [function words](https://docs.tdm-pilot.org/key-terms/#function-words), but that still may not give us results that describe the unique terms in the document since the uniqueness of terms depends on the context of a larger body of documents. In other words, the same term could be significant or insignificant depending on the context. Consider these examples:

* Given a set of scientific journal articles in biology, the term "lab" may not be significant since biologists often rely on and mention labs in their research. However, if the term "lab" were to occur frequently in a history or English article, then it is likely to be significant since humanities articles rarely discuss labs. 
* If we were to look at thousands of articles in literary studies, then the term "postcolonial" may be significant for any given article. However, if were to look at a few hundred articles on the topic of "the global south," then the term "postcolonial" may occur so frequently that it is not a significant way to differentiate between the articles.

The [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) calculation reveals the words that are frequent in this document **yet rare in other documents**. The goal is to find out what is unique or remarkable about a document given the context (and *the given context* can change the results of the analysis). 

Here is how the calculation is mathematically written:

$$tfidf_{t,d} = tf_{t,d} \cdot idf_{t,D}$$

In plain English, this means: **The value of [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) is the product (or multiplication) of a given term's frequency multiplied by its inverse document frequency.** Let's unpack these terms one at a time.

### Term Frequency Function

$$tf_{t,d}$$
The number of times (t) a term occurs in a given document (d)

### Inverse Document Frequency Function

$$idf_i = \mbox{log} \frac{N}{|{d : t_i \in d}|}$$
The inverse document frequency can be expanded to the calculation on the right. In plain English, this means: **The log of the total number of documents (N) divided by the number of documents that contain the term**

### TF-IDF Calculation in Plain English

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

There are variations on the [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) formula, but this is the most widely-used version.

### An Example Calculation of TF-IDF

Let's take a look at an example to illustrate the fundamentals of [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf). First, we need several texts to compare. Our texts will be very simple.

* text1 = 'The grass was green and spread out the distance like the sea.'
* text2 = 'Green eggs and ham were spread out like the book.'
* text3 = 'Green sailors were met like the sea met troubles.'
* text4 = 'The grass was green.'

The first step is we need to discover how many unique words are in each text. 

|text1|text2|text3|text4|
|    ---    | ---| --- | --- |
|the|green|green|the|
|grass|eggs|sailors|grass|
|was|and|were|was|
|green|ham|met|green|
|and|were|like| |
|spread|spread|the| |
|out|out|sea| |
|into|like|met| |
|distance|the|troubles| |
|like|book| | |
|sea| | | |


Our four texts share some similar words. Next, we create a single list of unique words that occur across all three texts. (When we use the [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) library later, we will call this list a [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary).) 

|Unique Words|
| --- |
|and|
|book|
|distance|
|eggs|
|grass|
|green|
|ham|
|like|
|met|
|out|
|sailors|
|sea|
|spread|
|the|
|troubles|
|was|
|were|

Now let's count the occurences of each unique word in each sentence

|word|text1|text2|text3|text4|
|---|---|---|---|---|
|and|1|1|0|0|
|book|0|1|0|0|
|distance|1|0|0|0|
|eggs|0|1|0|0|
|grass|1|0|0|1|
|green|1|1|1|1|
|ham|0|1|0|0|
|like|1|1|1|0|
|met|0|0|2|0|
|out|1|1|0|0|
|sailors|0|0|1|0|
|sea|1|0|1|0|
|spread|1|1|0|0|
|the|3|1|1|1|
|troubles|0|0|1|0|
|was|1|0|0|1|
|were|0|1|1|0|

### Computing TF-IDF (Example 1)

We have enough information now to compute [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) for every word in our corpus. Recall the plain English formula.

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

We can use the formula to compute [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) for the most common word in our corpus: 'the'. In total, we will compute [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) four times (once for each of our texts). 

|word|text1|text2|text3|text4|
|---|---|---|---|---|
|the|3|1|1|1|

text1: $$ tf-idf = 3 \cdot \mbox{log} \frac{4}{(4)} = 3 \cdot \mbox{log} 1 = 3 \cdot 0 = 0$$
text2: $$ tf-idf = 1 \cdot \mbox{log} \frac{4}{(4)} = 1 \cdot \mbox{log} 1 = 1 \cdot 0 = 0$$
text3: $$ tf-idf = 1 \cdot \mbox{log} \frac{4}{(4)} = 1 \cdot \mbox{log} 1 = 1 \cdot 0 = 0$$
text4: $$ tf-idf = 1 \cdot \mbox{log} \frac{4}{(4)} = 1 \cdot \mbox{log} 1 = 1 \cdot 0 = 0$$

The results of our analysis suggest 'the' has a weight of 0 in every document. The word 'the' exists in all of our documents, and therefore it is not a significant term to differentiate one document from another.

Given that idf is

$$\mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

and 

$$\mbox{log} 1 = 0$$
we can see that [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) will be 0 for any word that occurs in every document. That is, if a word occurs in every document, then it is not a significant term for any individual document.



### Computing TF-IDF (Example 2)

Let's try a second example with the word 'out'. Recall the plain English formula.

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

We will compute [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) four times, once for each of our texts. 

|word|text1|text2|text3|text4|
|---|---|---|---|---|
|out|1|1|0|0|

text1: $$ tf-idf = 1 \cdot \mbox{log} \frac{4}{(2)} = 1 \cdot \mbox{log} 2 = 1 \cdot .3010 = .3010$$
text2: $$ tf-idf = 1 \cdot \mbox{log} \frac{4}{(2)} = 1 \cdot \mbox{log} 2 = 1 \cdot .3010 = .3010$$
text3: $$ tf-idf = 0 \cdot \mbox{log} \frac{4}{(2)} = 0 \cdot \mbox{log} 2 = 0 \cdot .3010 = 0$$
text4: $$ tf-idf = 0 \cdot \mbox{log} \frac{4}{(2)} = 0 \cdot \mbox{log} 2 = 0 \cdot .3010 = 0$$

The results of our analysis suggest 'out' has some significance in text1 and text2, but no significance for text3 and text4 where the word does not occur.

### Computing TF-IDF (Example 3)

Let's try one last example with the word 'met'. Here's the [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) formula again:

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

And here's how many times the word 'met' occurs in each text.

|word|text1|text2|text3|text4|
|---|---|---|---|---|
|met|0|0|2|0|

text1: $$ tf-idf = 0 \cdot \mbox{log} \frac{4}{(1)} = 0 \cdot \mbox{log} 4 = 1 \cdot .6021 = 0$$
text2: $$ tf-idf = 0 \cdot \mbox{log} \frac{4}{(1)} = 0 \cdot \mbox{log} 4 = 1 \cdot .6021 = 0$$
text3: $$ tf-idf = 2 \cdot \mbox{log} \frac{4}{(1)} = 2 \cdot \mbox{log} 4 = 1 \cdot .6021 = 1.2042$$
text4: $$ tf-idf = 0 \cdot \mbox{log} \frac{4}{(1)} = 0 \cdot \mbox{log} 4 = 1 \cdot .6021 = 0$$

As should be expected, we can see that the word 'met' is very significant in text3 but not significant in any other text since it does not occur in any other text. 

## The Full TF-IDF Example Table

Here are the original sentences for each text:

* text1 = 'The grass was green and spread out the distance like the sea.'
* text2 = 'Green eggs and ham were spread out like the book.'
* text3 = 'Green sailors were met like the sea met troubles.'
* text4 = 'The grass was green.'

And here's the corresponding [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) scores for each word in each text:

|word|text1|text2|text3|text4|
|---|---|---|---|---|
|and|.3010|.3010|0|0|
|book|0|.6021|0|0|
|distance|.6021|0|0|0|
|eggs|0|.6021|0|0|
|grass|.3010|0|0|.3010|
|green|0|0|0|0|
|ham|0|.6021|0|0|
|like|.1249|.1249|.1249|0|
|met|0|0|.6021|0|
|out|.3010|.3010|0|0|
|sailors|0|0|.6021|0|
|sea|.3010|0|.3010|0|
|spread|.3010|.3010|0|0|
|the|0|0|0|0|
|troubles|0|0|.6021|0|
|was|.3010|0|0|.3010|
|were|0|.3010|.3010|0|

There are a few noteworthy things in this data. 

* The [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) score for any word that does not occur in a text is 0.
* The scores for almost every word in text4 are 0 since it is a shorter version of text1. There are no unique words in text4 since text1 contains all the same words. It is also a short text which means that there are only four words to consider. The words 'the' and 'green' occur in every text, leaving only 'was' and 'grass' which are also found in text1.
* The words 'book', 'eggs', and 'ham' are significant in text2 since they only occur in that text.

Now that you have a basic understanding of how [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) is computed at a small scale, let's try computing [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) on a [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) which could contain millions of words.

---

# Computing TF-IDF with your JSTOR/Portico Dataset

## Import your dataset

We'll use the `tdm_client` library to automatically upload your dataset. We import the `Dataset` module from the `tdm_client` library. The tdm_client library contains functions for connecting to the JSTOR server containing the [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset). To analyze your dataset, use the [dataset ID](https://docs.tdm-pilot.org/key-terms//#dataset-ID) provided when you created your [dataset](https://docs.tdm-pilot.org/key-terms//#dataset). A copy of your [dataset ID](https://docs.tdm-pilot.org/key-terms//#dataset-ID) was sent to your email when you created your [corpus](https://docs.tdm-pilot.org/key-terms/#corpus). It should look like a long series of characters surrounded by dashes. If you haven't created a dataset, feel free to use a sample dataset. Here's a [list by discipline](https://docs.tdm-pilot.org/sample-datasets/). Advanced users can also [upload a dataset from their local machine](https://docs.tdm-pilot.org/uploading-a-dataset/).

In [6]:
#Importing your dataset with a dataset ID
import tdm_client
tdm_client.get_dataset("f6ae29d4-3a70-36ee-d601-20a8c0311273", "sampleJournalAnalysis") #Load the sample dataset, the full run of Shakespeare Quarterly from 1950-2013.

HTTPError: HTTP Error 502: Bad Gateway

Before we can begin working with our [dataset](https://docs.tdm-pilot.org/key-terms/#dataset), we need to convert the [JSON lines](https://docs.tdm-pilot.org/key-terms/#jsonl) file format into [Python](https://docs.tdm-pilot.org/key-terms/#python) so we can work with it. Remember that each line of our [JSON lines](https://docs.tdm-pilot.org/key-terms/#jsonl) file represents a single text, whether that is a journal article, book, or something else. We will create a [Python](https://docs.tdm-pilot.org/key-terms/#python) list that contains every document. Within each list item for each document, we will use a [Python dictionary](https://docs.tdm-pilot.org/key-terms/#python-dictionary) of [key/value pairs](https://docs.tdm-pilot.org/key-terms/#key-value-pair) to store information related to that document. [Read more about the dataset format](https://docs.tdm-pilot.org/what-format-are-jstor-portico-datasets/).

Essentially we will have a [list](https://docs.tdm-pilot.org/key-terms/#python-list) of documents numbered, from zero to the last document. Each [list](https://docs.tdm-pilot.org/key-terms/#python-list) item then will be composed of a [dictionary](https://docs.tdm-pilot.org/key-terms/#python-dictionary) of [key/value pairs](https://docs.tdm-pilot.org/key-terms/#key-value-pair) that allows us to retrieve information from that particular document by number. The structure will look something like this:

![Structure of the corpus, a list of dictionaries](https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CorpusView.png)

For each item in our list we will be able to use [key/value pairs](https://docs.tdm-pilot.org/key-terms/#key-value-pair) to get a **value** if we supply a **key**. We will call our [Python list](https://docs.tdm-pilot.org/key-terms/#python-list) variable `all_documents` since it will contain all of the documents in our [corpus](https://docs.tdm-pilot.org/key-terms/#corpus).

In [ ]:
# Replace with your filename and be sure your file is in your datasets folder
file_name = 'sampleJournalAnalysis.jsonl' 

# Import the json module
import json
# Create an empty new list variable named `all_documents`
all_documents = [] 
# Temporarily open the file `filename` in the datasets/ folder
with open('./datasets/' + file_name) as dataset_file: 
    #for each line in the dataset file
    for line in dataset_file: 
        # Read each line into a Python dictionary.
        # Create a variable document that contains the line using json.loads to convert the json key/value pairs to a python dictionary
        document = json.loads(line) 
        # Append a new list item to `all_documents` containing the dictionary we created.
        all_documents.append(document) 

Now all of our documents have been converted from our original [JSON lines](https://docs.tdm-pilot.org/key-terms/#jsonl) file format (.jsonl) into a [python List](https://docs.tdm-pilot.org/key-terms/#python-list) variable named `all_documents`. Let's see what we can discover about our [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) with a few simple methods.

First, we can determine how many texts are in our [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) by using the `len()` function to get the size of `all_documents`. 

In [ ]:
len(all_documents)

---
## Removing Articles That Are Not Full-Length

When journal articles are added to JSTOR, they are broken up into chunks called articles. If we want to analyze every word in every issue, this approach works well. However, if we only want to analyze the full-length articles, we may want to remove some articles. These articles could be things like:

* Tables of Contents
* Indices
* Unauthored Materials
* Short Notes
* Book Reviews

We can design a set of assessments that will remove these materials. Depending on the journal and the field, shorter articles may still be relevant. The example code below demonstrates a set of rules for filtering using the article metadata.

### Articles To Be Removed
* Articles not in English
* Articles with no authors
* Articles with title: "Review Article"
* Articles with title: "Front Matter"
* Articles with title: "Back Matter"
* Articles with a word count less than 3000 words

The function below outputs a list of the first ten articles followed by the reason each article would or would not be kept. This can be a useful exploratory tool for getting the best starting corpus. You may need to adjust the word count number up or down, particularly for distinguishing between say short notes and full-length articles. You might also consider writing other [metadata](https://docs.tdm-pilot.org/key-terms/#metadata) field tests to narrow your [corpus](https://docs.tdm-pilot.org/key-terms/#corpus). **Note: The Article Assessment Exploration code below does not change your corpus found within ``all_documents``. It merely serves as a convenient way to adjust the logic of the actual filtering that happens in the next step.**

In [ ]:
#Article Assessment Exploration

## Define a function ``remove_non_articles`` that will test a single document
def remove_non_articles(test_doc):
    print('Article ' + str(i) + ':') # Print the list index for each article so they can be easily referenced
    print('Title: ' + test_doc.get('title')) # Print the title for the article in question
    print('URL: ' + test_doc.get('id')) # Print the URL for the article so it can be quickly reviewed
    print('Status: ', end='') # Print the phrase 'Status' without a following line-break
    #if test_doc.get('language') != ['eng']:
        #print('Removed--Not in English')
    if test_doc.get('creators') == None: # Get the value for the key 'creators' in the test document and check if it is equal to none
        print('Removed--No author') # If the value for 'creators' is none, print 'Removed--No author'
    elif test_doc.get('title') == 'Review Article': # Get the value for the key 'title' in the test document and check if it is equal to 'Review Article'
        print('Removed--Review Article') # If the value for 'title' is 'Review Article', print 'Removed--Review Article'
    elif test_doc.get('title') == 'Front Matter': # Get the value for the key 'title' in the test document and check if it is equal to 'Front Matter'
        print('Removed--Front Matter') # If the value for 'title' is 'Front Matter', print 'Removed--Front Matter'
    elif test_doc.get('title') == 'Back Matter': # Get the value for the key 'title' in the test document and check if it is equal to 'Back Matter'
        print('Removed--Back Matter')  # If the value for 'title' is 'Back Matter', print 'Removed--Back Matter'
    elif test_doc.get('wordCount') < 3000: # Get the value for 'wordCount' in the test document and check if the integer is less than 3000 words (Change this number if you want more or less words)
        print('Removed--Too short at ' + str(test_doc.get('wordCount')) + ' words') # If the value for wordCount is less than 3000, print 'Removed--Too short at' followed by the actual word count
    else:
        print('GOOD ARTICLE at '+ str(test_doc.get('wordCount')) + ' words') # If the article passes all the above tests, print 'GOOD ARTICLE at ' with the article word count

articles_to_show = 20 # Show the first ten articles (Change this number to show more or fewer articles)
#articles_to_show = len(all_documents) # Uncomment to show all articles
for i in range(articles_to_show): # Repeat this process the number of times as the value of ``articles_to_show`` + 1
    remove_non_articles(all_documents[i])  # Run the remove_non_articles function on a single document with list index value of ``i``

Now that we've done some exploratory analysis to figure out the right parameters for filtering our corpus, we can put them into actual practice. The following set of [list comprehensions](https://docs.tdm-pilot.org/key-terms/#list-comprehensions) creates a new list called ``reduced_list`` from our original [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) ``all_documents``. These [list comprehensions](https://docs.tdm-pilot.org/key-terms/#list-comprehensions) can be adjusted to your [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) to get the best results. For example, you may want to change the ``wordCount`` to be larger or smaller than the default below.

After each list reduction, the number of remaining articles is printed for reference.

In [ ]:
print('Original number of documents: ' + str(len(all_documents))) # Print the original number of documents in ``all_documents``

reduced_list = []
reduced_list = [all_documents[x] for x in range(len(all_documents)) if all_documents[x].get('language') == ['eng']] # Copy each list item from ``all_documents`` to ``reduced_list`` if the ``language`` key has a value pair of ['eng']
print('After removing articles not in English: ' + str(len(reduced_list))) # Print the current size of ``reduced_list``
#Note that this first filter works on ``all_documents`` but the following filters must work on ``reduced_list`` 

reduced_list = [reduced_list[x] for x in range(len(reduced_list)) if reduced_list[x].get('creators') != None] # Copy each list item from ``reduced_list`` to ``reduced_list`` if the ``creators`` key does not have a value pair of None
print('After removing articles with no authors: ' + str(len(reduced_list))) # Print the current size of ``reduced_list``

reduced_list = [reduced_list[x] for x in range(len(reduced_list)) if reduced_list[x].get('title') != 'Review Article'] # Copy each list item from ``reduced_list`` to ``reduced_list`` if the ``title`` key does not have a value pair of 'Review Article'
print('After removing "Review Articles": ' + str(len(reduced_list))) # Print the current size of ``reduced_list``

reduced_list = [reduced_list[x] for x in range(len(reduced_list)) if reduced_list[x].get('title') != 'Front Matter'] # Copy each list item from ``reduced_list`` to ``reduced_list`` if the ``title`` key does not have a value pair of 'Front Matter'
print('After removing articles labeled "Front Matter": ' + str(len(reduced_list))) # Print the current size of ``reduced_list``

reduced_list = [reduced_list[x] for x in range(len(reduced_list)) if reduced_list[x].get('title') != 'Back Matter'] # Copy each list item from ``reduced_list`` to ``reduced_list`` if the ``title`` key does not have a value pair of 'Back Matter'
print('After removing articles labeled "Back Matter": ' + str(len(reduced_list))) # Print the current size of ``reduced_list``

reduced_list = [reduced_list[x] for x in range(len(reduced_list)) if reduced_list[x].get('wordCount') > 3000] # Copy each list item from ``reduced_list`` to ``reduced_list`` if the ``wordCount`` has a value pair less than 3000
print('After removing short articles: ' + str(len(reduced_list))) # Print the current size of ``reduced_list``

If you'd like to see what's in your ``reduced_list``, you can run the code below to supply some of the metadata for the first 10 items.

In [ ]:
#Print information for the first 10 items in the ``reduced_list`` 

for r in range (10):
    print('Title ' + str(r) + ': ' + reduced_list[r].get('title'))
    print('Language: ' + str(reduced_list[r].get('language')))
    print('Authors: ' + str(reduced_list[r].get('creators')))
    print('Number of words: ' + str(reduced_list[r].get('wordCount')), end='\n\n')

## Cleaning Up the Tokens in the Corpus

Let's create a helper function that can standardize and [clean](https://docs.tdm-pilot.org/key-terms/#clean-data) up the [tokens](https://docs.tdm-pilot.org/key-terms/#token) in our [dataset](https://docs.tdm-pilot.org/key-terms/#dataset). The function will:
* lower case all [tokens](https://docs.tdm-pilot.org/key-terms/#token)
* use a dictionary from [The HathiTrust Research Center](https://docs.tdm-pilot.org/key-terms/#htrc) to correct common [Optical Character Recognition](https://docs.tdm-pilot.org/key-terms/#ocr) problems
* discard [tokens](https://docs.tdm-pilot.org/key-terms/#token) less than 4 characters in length
* discard [tokens](https://docs.tdm-pilot.org/key-terms/#token) with non-alphabetical characters
* remove [stopwords](https://docs.tdm-pilot.org/key-terms/#stop-words) based on [The HathiTrust Research Center](https://docs.tdm-pilot.org/key-terms/#htrc) [stopword](https://docs.tdm-pilot.org/key-terms/#stop-words) list

In [ ]:
from tdm_client import htrc_corrections # Import the htrc_corrections that helps correct common OCR problems

def process_token(token): #define a function `process_token` that takes the argument `token`
    token = token.lower() #set the string in token to a new string with all lowercase letters
    corrected = htrc_corrections.get(token) #initialize a new variable `corrected` that runs token through the `htrc_corrections.get()` function to fix common OCR errors
    if corrected is not None: #if corrected has a value, set the `token` variable to the same value as `corrected`
        token = corrected
    if len(token) < 4: #if token is less than four characters, return nothing for process_function (no output here essentially erases this token)
        return
    if not(token.isalpha()): #if token contains non-alphabetic characters, return nothing for process_function (no output here essentially erases this token)
        return
    return token #return the `token` variable which has been set equal to the `corrected` variable

def process_document(chosen_document): # Create a new function ``process_document`` that takes the argument chosen_document
    this_doc = [] # Create a new list ``this_doc`` that will hold the contents of the current document
    singleDoc = chosen_document.get('unigramCount') # Create a list variable ``singleDoc` that will contain the contents of `unigramCount` for the current document
    for token, count in singleDoc.items(): # For each token in the document, 
        clean_token = process_token(token) # Use the ``process_token`` function above to clean that token
        if clean_token is None: # If there is no token returned, proceed
            continue
        this_doc += [clean_token] * count # Add to ``this_doc`` list the number of token occurences
    documents.append(this_doc) # Add the token count results for ``this_doc`` to the ``documents`` list

Now let's cycle through each document in the [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) with our helper function.

In [ ]:
documents = [] # An empty variable ``documents`` that will contain all our documents with cleaned tokens
for i in range(len(reduced_list)): # Repeat this process once for every document in ``reduced_list``
    process_document(reduced_list[i]) # Run the ``process_token`` function on the single article by reference to its index number of i

---
# Using Gensim to Compute "Term Frequency- Inverse Document Frequency"

It will be helpful to remember the basic steps we did in the explanatory [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) example:

1. Create a list of the frequency of every word in every document
2. Create a list of every word in the [corpus](https://docs.tdm-pilot.org/key-terms/#corpus)
3. Compute [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) based on that data

So far, we have completed the first item by creating a list of the frequency of every word in every document. Now we need to create a list of every word in the corpus. In [gensim](https://docs.tdm-pilot.org/key-terms/#gensim), this is called a "dictionary". A [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) is similar to a [Python dictionary](https://docs.tdm-pilot.org/key-terms/#python-dictionary), but here it is called a [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) to show it is a specialized kind of dictionary.

## Creating a Gensim Dictionary

Let's create our [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary). A [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) is a kind of masterlist of all the words across all the documents in our corpus. Each unique word is assigned an ID in the gensim dictionary. The result is a set of key/value pairs of unique tokens and their unique IDs.

In [ ]:
import gensim
dictionary = gensim.corpora.Dictionary(documents) # Create the Gensim dictionary based on our ``documents`` variable

Now that we have a [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary), we can get a preview that displays the number of unique tokens across all of our texts.

In [ ]:
print(dictionary)

The [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) stores a unique identifier (starting with 0) for every unique token in the corpus. The [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) does not contain information on word frequencies; it only catalogs all the words in the corpus. You can see the unique ID for each token in the text using the .token2id() method. Your corpus may have hundreds of thousands of unique words so here we just give a preview of the first ten.

In [ ]:
dict(list(dictionary.token2id.items())[0:10]) # Print the first ten tokens and their associated IDs.


We can also look up the corresponding ID for a token using the ``.get`` method.

In [ ]:
dictionary.token2id.get('people', 0) # Get the value for the key 'people'. Return 0 if there is no token matching 'people'. The number returned is the gensim dictionary ID for the token. 

## Creating a Bag of Words Corpus


### A Single Document Example

The next step is to combine our word frequency data found within ``documents`` to our [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) token IDs. For every document, we want to know how many times a word (notated by its ID) occurs. We can do a single document first to show how this works. We will create a [Python list](https://docs.tdm-pilot.org/key-terms/#python-list) called ``example_bow_corpus`` that will turn our word counts into a series of [tuples](https://docs.tdm-pilot.org/key-terms/#tuple) where the first number is the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) token ID and the second number is the word frequency.

In [ ]:
example_bow_corpus = [dictionary.doc2bow(documents[31])] # Create an example bag of words corpus. We select a document at random to use as our sample.
list(example_bow_corpus[0][:10]) # List out the first ten tuples in ``example_bow_corpus``

Using IDs can seem a little abstract, but we can discover the word associated with a particular ID. For demonstration purposes, the following code will replace the token IDs in the last example with the actual tokens.

In [ ]:
word_counts = [[(dictionary[id], count) for id, count in line] for line in example_bow_corpus]
list(word_counts[0][:10])

We saw before that you could discover the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) ID number by running:

> dictionary.token2id.get('people', 0)

If you wanted to discover the token given only the ID number, the method is a little more involved. You could use [list comprehension](https://docs.tdm-pilot.org/key-terms/#list-comprehensions) to find the **key** token based on the **value** ID. Normally, [Python dictionaries](https://docs.tdm-pilot.org/key-terms/#python-dictionary) only map from keys to values (not from values to keys). However, we can write a quick [list comprehension](https://docs.tdm-pilot.org/key-terms/#list-comprehensions) to go the other direction. (It is unlikely one would ever do these methods in practice, but they are shown here to demonstrate how the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) is connected to the list entries in the [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) ``bow_corpus``. 

In [ ]:
[token for dict_id, token in dictionary.items() if dict_id == 155] # Find the corresponding token in our gensim dictionary for the gensim dictionary ID 239

## Creating a Bag of Words Corpus Using Every Document

We have seen an example that demonstrates how the [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) [bag of words](https://docs.tdm-pilot.org/key-terms/#bag-of-words) [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) works on a single document. Let's apply it now to all of our documents. 

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
#print(bow_corpus[:3]) #Show the bag of words corpus for the first 3 documents

The next step is to create the [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) model which will set the parameters for our implementation of [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf). In our [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) example, the formula for [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) was:

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

In [gensim](https://docs.tdm-pilot.org/key-terms/#gensim), the default formula for measuring [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) uses log base 2 instead of log base 10, as shown:

$$(Times-the-word-occurs-in-given-document) \cdot \log_{2} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-the-word)}$$

If you would like to use a different formula for your [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) calculation, there is a description of [parameters you can pass](https://radimrehurek.com/gensim/models/tfidfmodel.html).

In [ ]:
model = gensim.models.TfidfModel(bow_corpus) # Create our gensim TF-IDF model

Now, we apply our model to the ``bow_corpus`` to create our results in ``corpus_tfidf``. The ``corpus_tfidf`` is a python list of each document similar to ``bow_document``. Instead of listing the frequency next to the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) ID, however, it contains the TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) score for the associated token. Below, we display the first document in ``corpus_tfidf``. 

In [ ]:
corpus_tfidf = model[bow_corpus] # Create TF-IDF scores for the ``bow_corpus`` using our model
list(corpus_tfidf[0][:10]) # List out the TF-IDF scores for the first 10 tokens of the first text in the corpus

Let's display the tokens instead of the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) IDs.

In [ ]:
example_tfidf_scores = [[(dictionary[id], count) for id, count in line] for line in corpus_tfidf]
list(example_tfidf_scores[0][:10]) # List out the TF-IDF scores for the first 10 tokens of the first text in the corpus

Finally, let's sort the terms by their [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) weights to find the most significant terms in the document.

In [ ]:
# Sort the tuples in our tf-idf scores list

def Sort(tfidf_tuples): 
    tfidf_tuples.sort(key = lambda x: x[1], reverse=True) 
    return tfidf_tuples 

list(Sort(example_tfidf_scores[0])[:10]) #List the top ten tokens in our example document by their TF-IDF scores

We could also analyze across the entire corpus to find the most unique terms. These are terms that appear frequently in a single text, but rarely or never appear in other texts. (Often, these will be proper names since a particular article may mention a name often but the name may rarely appear in other articles.)

In [ ]:
td = { # Define a dictionary ``td`` where each document gather
        dictionary.get(_id): value for doc in corpus_tfidf
        for _id, value in doc
    }
sorted_td = sorted(td.items(), key=lambda kv: kv[1], reverse=True) # Sort the items of ``td`` into a new variable ``sorted_td``, the ``reverse`` starts from highest to lowest

In [ ]:
for term, weight in sorted_td[:25]: # Print the top 25 terms in the entire corpus
    print(term, weight)

And, finally, we can see the most significant term in every document.

In [ ]:
for n, doc in enumerate(corpus_tfidf):
    if len(doc) < 1:
        continue
    word_id, score = max(doc, key=lambda x: x[1])
    print(reduced_list[n].get('id'), dictionary.get(word_id), score)
    if n >= 10:
        break

____
# Start Next Lesson: [LDA Topic Modeling](./1-topic-modeling.ipynb)